<h1 style="text-align: center">Implement Multi-Perception</h1>

# Import lib

In [1]:
import unittest
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

## Tính toán đầu ra của mạng (tính xuôi)

Lặp $L$ lần, quy ước $\mathbf f_0 = \mathbf x$, với $l = 1,2,\ldots L$

$$ \begin{align*} \mathbf a_l &= \mathbf W_l \mathbf f_{l-1}\in\mathbb R^{p_l}\\ \mathbf f_l &= \phi_l(\mathbf a_l)\in\mathbb R^{p_l} \end{align*} $$

Tính logit, xác suất (softmax) và hàm lỗi cross-entropy

$$ \begin{align*} \mathbf f &= \mathbf f_L \in\mathbb R^{C}\\ \bf\mu &=\mathcal S(\mathbf f)\in\mathbb R^{C}\\ \ell &= -\mathbf y^T\log(\bf\mu)\in\mathbb R \end{align*} $$

$\mathbf y$ là **mã hoá one-hot** của nhãn $y\in\{1,2,\ldots, C\}$ .

Mạng nơ-ron có $L-1$ lớp ẩn và một lớp đầu ra.

## Lựa chọn hàm kích hoạt

- Hàm tuyến tính: $\phi_l(a) = a$, hay dùng ở lớp cuối cùng
- Hàm sigmoid: $\phi_l(a) = \sigma(a) = \frac 1 {1+e^{-a}}$, hay dùng ở các lớp trước lớp cuối cùng (lớp ẩn)
- Hàm ReLU: $\phi_l(a) = \max(0, a)$
- Hàm tanh: $\phi_l(a) = 2\sigma(a)-1$

💡 Nếu chọn $\phi_l(a) = a$ với mọi tầng của mạng thì sẽ có lại Hồi quy Logistics (trường hợp con của mạng nơ-ron)



## Suy luận bằng mạng nơ-ron

**Luật phân lớp**: chọn vị trí phần tử lớn nhất trong $\mathbf f$ là phân lớp của $\mathbf x$.

## Kích thước của bộ trọng số

- Đầu ra của tầng trước là đầu vào của tầng sau: ma trận $\mathbf W_l\in \mathbb R^{p_l\times p_{l-1}}$, trong đó $p_l$ là số đầu ra của lớp $l$ còn $p_{l-1}$ là số đầu ra của lớp $l-1$.
- Quy ước $p_0 = d+1$ là số đầu vào của mạng.
- Lớp cuối cùng: $p_L = C$ là số lớp của bài toán phân lớp.

In [86]:
class TestFullyConnected(unittest.TestCase):
    
    def test_fc_init(self):
        fc = FC(n_in = 3, n_out = 5, activation = "sigmoid")
        self.assertEqual(fc.n_in, 3)
        self.assertEqual(fc.n_out, 5)
        self.assertEqual(fc.activation, "sigmoid")
        self.assertEqual(fc.W.shape, (3, 5))
        self.assertEqual(fc.dW.shape, (3, 5))
    
    def test_fc_forward(self):
        fc = FC(n_in = 3, n_out = 5, activation = "sigmoid")
        x = np.zeros((10, 3), dtype=np.float32)
        y = fc.forward(x)
        error = np.sum(np.abs((y - np.ones_like(y) * 0.5)))
        self.assertEqual(y.shape, (10, 5))
        self.assertLess(error, 1e-6)

    def test_fc_forward_identity(self):
        fc = FC(n_in = 3, n_out = 5, activation = None)
        x = np.zeros((10, 3), dtype=np.float32)
        y = fc.forward(x)
        error = np.sum(np.abs(y - np.zeros_like(y)))
        self.assertEqual(y.shape, (10, 5))
        self.assertLess(error, 1e-6)
        
    def test_fc_backward(self):
        fc = FC(n_in = 3, n_out = 5, activation = "sigmoid")
        x = np.zeros((10, 3), dtype=np.float32)
        y = fc.forward(x)
        dx = fc.backward(np.zeros_like(y))
        self.assertEqual(dx.shape, x.shape)
        self.assertEqual(fc.dW.shape, fc.W.shape)
    
    def test_fc_backward_identity(self):
        fc = FC(n_in = 3, n_out = 5, activation = None)
        x = np.zeros((10, 3), dtype=np.float32)
        y = fc.forward(x)
        dx = fc.backward(np.zeros_like(y))
        self.assertEqual(dx.shape, x.shape)
        self.assertEqual(fc.dW.shape, fc.W.shape)
    

## Đạo hàm của hàm lỗi (tính ngược)

### B1. Tính đạo hàm $\delta_{\bf \mu}$

$$ \delta_{\bf\mu} = -\mathbf y^T / \bf\mu^T\in\mathbb R^{1\times C} $$

<aside> 💡 Lưu ý: vector hàng (dòng)

</aside>

### B2. Tính đạo hàm $\mathbf J_{\bf\mu}(\mathbf f)$

$$ \begin{align*} \frac{\partial \mu_i}{\partial f_j} &= \frac{u'v-v'u}{v^2} =\frac{\mathbb I(i=j) e^{f_j}\sum_{c=1}^C e^{f_c}-e^{f_j}e^{f_i}}{(\sum_{c=1}^C e^{f_c})^2}\\ &= \mathbb I(i=j) \mu_j - \mu_i\mu_j=\begin{cases}(1-\mu_i)\mu_i&i=j\\-\mu_i\mu_j&i\neq j\end{cases}\\&=\begin{bmatrix}(1-\mu_1)\mu_1 &-\mu_2\mu_1 &\cdots&-\mu_K\mu_1\\ -\mu_1\mu_2&(1-\mu_2)\mu_2 &\cdots&-\mu_K\mu_2\\\vdots&\vdots&&\vdots\\-\mu_1\mu_K &-\mu_2\mu_K &\cdots&(1-\mu_K)\mu_K\end{bmatrix}\in\mathbb R^{C\times C} \end{align*} $$

### B3. Tính đạo hàm $\delta_{\mathbf f}$

$$ \delta_{\mathbf f_L=}\delta_{\mathbf f} = \delta_{\bf\mu}\mathbf J_{\bf\mu}(\mathbf f)\in\mathbb R^{1\times C} $$

Trong trường softmax và cross-entropy thì $\delta_{\mathbf f} = \bf\mu^T-\mathbf y^T$.

### B4. Tính đạo hàm $\mathbf J_{\mathbf f_l}(\mathbf a_l)$

Hàm tuyến tính $\phi_l(a) = a$ thì $\mathbf J_{\mathbf f_l}(\mathbf a_l)=\mathbf I\in\mathbb R^{p_l\times p_l}$

Hàm sigmoid $\phi_l(a) = \sigma(a)$ thì $\mathbf J_{\mathbf f_l}(\mathbf a_l)=\mathrm{diag}(f_{l1}(1-f_{l1}),f_{l2}(1-f_{l2}),\ldots,f_{lp_l}(1-f_{lp_l}))$

 💡 Do hàm kích hoạt được tính trên từng phần tử của $\mathbf a_l$ nên ma trận $\mathbf J_{\mathbf f_l}(\mathbf a_l)$ là ma trận đường chéo

### B5. Tính đạo hàm $\mathbf J_{\mathbf a_l}(\mathbf f_{l-1})$

$$ \mathbf a_l = \mathbf W_l \mathbf f_{l-1} $$

$$ \mathbf J_{\mathbf a_l}(\mathbf f_{l-1}) = \mathbf W_l $$

### B6. Tính đạo hàm của $\mathbf a_l$ đối với $\mathbf W_l$.

$$ \frac {\partial a_{li}}{\partial\mathbf W_l}=\begin{bmatrix}0 &0 &\cdots&0\\\vdots&\vdots&&\vdots\\ -&\mathbf f_{l-1}^T&-&-\\\vdots&\vdots&&\vdots\\0&0&\cdots&0\end{bmatrix}\in\mathbb R^{p_l\times p_{l-1}} $$

Ma trận gồm toàn các dòng số 0, duy nhất dòng thứ $i$ là đầu vào $\mathbf f_{l-1}^T$.

### B7. Tính đạo hàm $\delta_{\mathbf a_l}, \delta_{\mathbf W_l}, \delta_{\mathbf f_{l-1}}$

$$ \begin{align*} \delta_{\mathbf a_l} &= \delta_{\mathbf f_l}\mathbf J_{\mathbf f_l}(\mathbf a_l)\in\mathbb R^{1\times p_l}\\ \delta_{\mathbf W_l} &= \delta_{\mathbf a_l}^T\mathbf f_{l-1}^T \in\mathbb R^{p_l\times p_{l-1}}\\ \delta_{\mathbf f_{l-1}}&=\delta_{\mathbf a_l}\mathbf W_l\in\mathbb R^{1\times p_{l-1}} \end{align*} $$

<aside> 💡 Công thức đầu tiên có ma trận đường chéo

</aside>

💡 Công thức thứ hai có được dựa vào đạo hàm thành phần $\frac{\partial \ell}{\partial\mathbf W_l}=\sum_{i=1}^{p_l}\frac{\partial\ell}{\partial a_{li}}\frac {\partial a_{li}}{\partial\mathbf W_l}$

In [90]:
class FC:
    def __init__(self, n_in, n_out, activation=None):
        self.n_in = n_in
        self.n_out = n_out
        
        self.activation = activation
        self.W = np.random.randn(n_in, n_out)
        self.dW = np.zeros_like(self.W, dtype = np.float32)
#         self.a_l = None
#         self.f_l = None
#         self.f_lb = None
        
        
    @staticmethod
    def stable_sigmoid(X):
        return np.where(X >= 0,
                        (1 + np.exp(-X))**-1,
                        np.exp(X) / (1 + np.exp(X)))
    
    # IForward
    def forward(self, X):
        assert(X.shape[1] == self.n_in)
        
        self.a_l = X @ self.W
        self.f_lb = X
        if self.activation == None:
            self.f_l = self.a_l.copy()
        elif self.activation == "sigmoid":
            self.f_l = self.stable_sigmoid(self.a_l)
        else: 
            raise NotImplementError(f"{self.activation} activation has been implemented yet")    
        return self.f_l
    
    def backward(self, output_grad):
        assert(output_grad.shape == self.f_l.shape)
        
        if self.activation == None:
            df = np.eye(self.n_out)
        elif self.activation == "sigmoid":
            df = np.diag((self.f_l * (1 - self.f_l)).ravel())
        else:
            raise NotImplementError(f"{self.activation} activation has been implemented yet")
        print(df)
        delta_a = output_grad @ df.T
        self.dW = delta_a.T @ self.f_lb.T
        
        return delta_a @ self.W        

In [85]:
fc = FC(n_in = 3, n_out = 5, activation='sigmoid')
x = np.ones((4, 3))
y = fc.forward(x)
fc.backward(np.ones_like(y))

[[0.12955634 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.07229678 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.02075926 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.14897406 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.2482604  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 20 is different from 5)

In [91]:
unittest.main(argv=[""], verbosity = 2, exit = False)

test_fc_backward (__main__.TestFullyConnected) ... ERROR
test_fc_backward_identity (__main__.TestFullyConnected) ... ERROR
test_fc_forward (__main__.TestFullyConnected) ... ok
test_fc_forward_identity (__main__.TestFullyConnected) ... ok
test_fc_init (__main__.TestFullyConnected) ... ok

ERROR: test_fc_backward (__main__.TestFullyConnected)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14684\2397475373.py", line 31, in test_fc_backward
    dx = fc.backward(np.zeros_like(y))
  File "C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14684\670710692.py", line 44, in backward
    delta_a = output_grad @ df.T
ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 50 is different from 5)

ERROR: test_fc_backward_identity (__main__.TestFullyConnected)
-------------------------------------------------------------------

[[0.25 0.   0.   ... 0.   0.   0.  ]
 [0.   0.25 0.   ... 0.   0.   0.  ]
 [0.   0.   0.25 ... 0.   0.   0.  ]
 ...
 [0.   0.   0.   ... 0.25 0.   0.  ]
 [0.   0.   0.   ... 0.   0.25 0.  ]
 [0.   0.   0.   ... 0.   0.   0.25]]
[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [75]:
a = np.array([[1,2], [3, 4]])
np.diag((a * (1 - a)).ravel())

array([[  0,   0,   0,   0],
       [  0,  -2,   0,   0],
       [  0,   0,  -6,   0],
       [  0,   0,   0, -12]])